# Constraining ΛCDM Parameters using Hubble Diagram Data (OHD)

## Workshop on Astrophysics and Cosmology – SXPC-NEPAL  
**Session:** Cosmology – Hubble Data Analysis  
**Date:** December 22, 2025  
**Presenter/Facilitator:** Subash Bhandari  

This notebook demonstrates how to:
- Load observational Hubble parameter data $H(z)$
- Implement the flat ΛCDM model
- Perform MCMC parameter estimation using `emcee`
- Visualise chains and posterior distributions with `corner`
- Report results in standard astrophysical format

---

## 1. Importing Libraries

In [ ]:
!pip install -r requirements.txt

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
from multiprocessing import Pool, cpu_count
import emcee
from tqdm import tqdm
import corner
import scipy.linalg as la
import arviz as az

## 2. Load Observational Data
Load the data file (ensure OHD.txt is in the same directory). The dataset contains three columns with redshift $z$, hubble measurements $H(z)$ and it's error $H_{err}$  

In [ ]:
data = np.loadtxt("OHD.txt")

z_data = data[:, 0]   # redshift
H_data = data[:, 1]   # H(z) in km/s/Mpc
H_err  = data[:, 2]   # 1σ uncertainty

print(f"Loaded {len(z_data)} data points.")
print("Redshift range: z = {:.2f} → {:.2f}".format(z_data.min(), z_data.max()))

## 3. Theoretical Model: Flat ΛCDM
For a flat universe:
$$H(z) = H_0 \sqrt{\Omega_m (1+z)^3 + (1-\Omega_m)}$$

In [ ]:
def H_model(z, params):
    """
    Flat ΛCDM Hubble parameter.
    
    Parameters
    ----------
    z : array-like
        Redshift
    params : list or array
        [Ω_m, H_0]
    """
    Om0, H0 = params
    return H0 * np.sqrt(Om0 * (1 + z)**3 + (1 - Om0))

## 4. Likelihood and χ² Definition

In [ ]:
def chi2(H_obs, H_th, sigma):
    """Simple χ² (assuming uncorrelated errors)"""
    return np.sum(((H_obs - H_th) / sigma)**2)

def log_likelihood(params):
    Om0, H0 = params
    
    # Priors (uniform)
    if not (0.0 < Om0 < 0.6 and 30 < H0 < 100):
        return -np.inf
    
    H_th = H_model(z_data, params)
    chi2_val = chi2(H_data, H_th, H_err)
    
    return -0.5 * chi2_val

## 5. Running MCMC with emcee

In [ ]:
# MCMC setup
nwalkers = 100
ndim     = 2
nsteps   = 10000

# Initial positions (uniform within priors)
p0 = np.random.uniform(low=[0.01, 50], high=[0.5, 90], size=(nwalkers, ndim))

# Use multiprocessing
ncpu = cpu_count()
print(f"Using {ncpu} CPUs")

with Pool(processes=ncpu) as pool:
    sampler = emcee.EnsembleSampler(nwalkers, ndim, log_likelihood, pool=pool)
    sampler.run_mcmc(p0, nsteps, progress=True)

## 6. Analyse Chains

In [ ]:
# Discard burn-in and thin
plt.rcParams.update({'font.size': 12})
burnin = 1000
thin   = 10

flat_samples = sampler.get_chain(discard=burnin, thin=thin, flat=True)
print(f"Final sample size: {flat_samples.shape[0]}")

# Median and 68% credible intervals
labels = [r"$\Omega_M$", r'$H_0$']

for i, label in enumerate(labels):
    mcmc = np.percentile(flat_samples[:, i], [16, 50, 84])
    q = np.diff(mcmc)
    print(f"{label} = {mcmc[1]:.3f}  +{q[1]:.3f}  -{q[0]:.3f}")

## 7. Corner Plots

In [ ]:
# Best-fit values for truth lines
truths = [np.median(flat_samples[:, i]) for i in range(ndim)]

fig = corner.corner(
    flat_samples,
    labels=labels,
    truths=truths,
    show_titles=True,
    title_fmt=".3f",
    levels=[0.68, 0.95],
    color="#1f77b4",
    smooth=1.5,
    fill_contours=True,
    plot_datapoints=False
)

plt.suptitle("Posterior Constraints on Flat ΛCDM Parameters", y=1.02, fontsize=16)
plt.show()

## 9. Best Fit Model vs Data

In [ ]:
# Best-fit parameters (median)
Om_best, H0_best = np.median(flat_samples, axis=0)

z_th = np.linspace(0, 2.5, 1000)
H_th = H_model(z_th, [Om_best, H0_best])

plt.figure(figsize=(9, 6));
plt.errorbar(z_data, H_data, yerr=H_err, fmt="o", capsize=4, label="OHD data")
plt.plot(z_th, H_th, color="red", lw=2, label=f"Best-fit: $\Omega_m={Om_best:.3f} $, $ H_0={H0_best:.1f} $");

plt.xlabel("Redshift $z$")
plt.ylabel("$ H(z) $ [km s$ ^{-1} $ Mpc$ ^{-1} $]")
plt.title("Hubble Diagram Fit with Flat ΛCDM")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout();
plt.show();

## 10. Summary & Discussion Points for Workshop

The obtained $\Omega_m$ and $H_0$ are consistent (within uncertainties) with Planck 2018 and SH0ES results, illustrating the Hubble tension when using different datasets. OHD measurements are model-independent probes of expansion history.
Extensions:
* Add spatial curvature ($Ω_k \neq 0$)
* Include BAO or CMB distance priors
* Compare with Pantheon+ supernovae or DESI BAO results